In [1]:
# dev of multi sigma per neuron

In [2]:
# case 0 - one sigma per neuron = sigma*identity covariance matrix

In [3]:
# case 1 - diag covariance matrix

In [4]:
import torch
import numpy as np

In [5]:
batch_size = 5
input_dim = 3
num_neurons = 2
inputs = torch.range(start=1, end=batch_size*input_dim).reshape((batch_size,input_dim))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  after removing the cwd from sys.path.


In [6]:
print("inputs")
print(inputs.shape)
print(inputs)
# inputs = torch.ones(size=(batch_size,input_dim))

inputs
torch.Size([5, 3])
tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.],
        [13., 14., 15.]])


In [7]:
# centers = torch.rand(num_neurons, input_dim)
centers = torch.range(start=1, end=num_neurons*input_dim).reshape((num_neurons,input_dim))
print("centers")
print(centers.shape)
print(centers)

centers
torch.Size([2, 3])
tensor([[1., 2., 3.],
        [4., 5., 6.]])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  


In [8]:
distances = inputs.unsqueeze(1)-centers
print("distances")
print(distances.shape)
print(distances)

distances
torch.Size([5, 2, 3])
tensor([[[ 0.,  0.,  0.],
         [-3., -3., -3.]],

        [[ 3.,  3.,  3.],
         [ 0.,  0.,  0.]],

        [[ 6.,  6.,  6.],
         [ 3.,  3.,  3.]],

        [[ 9.,  9.,  9.],
         [ 6.,  6.,  6.]],

        [[12., 12., 12.],
         [ 9.,  9.,  9.]]])


In [9]:
# distances per neuron
for n in range(num_neurons):
    print("neuron", n)
    print(distances[:,n,:].shape)
    print(distances[:,n,:])

('neuron', 0)
torch.Size([5, 3])
tensor([[ 0.,  0.,  0.],
        [ 3.,  3.,  3.],
        [ 6.,  6.,  6.],
        [ 9.,  9.,  9.],
        [12., 12., 12.]])
('neuron', 1)
torch.Size([5, 3])
tensor([[-3., -3., -3.],
        [ 0.,  0.,  0.],
        [ 3.,  3.,  3.],
        [ 6.,  6.,  6.],
        [ 9.,  9.,  9.]])


In [10]:
# create a diagonal matrix of random covariances 
covar_matrix = np.eye(input_dim)
covar_matrix = np.arange(1,input_dim+1)*covar_matrix
print("covar shape", covar_matrix.shape)
print(covar_matrix)
# covar_matrix for each neuron

covar_matrixes = torch.tensor(np.tile(A=covar_matrix,reps=(num_neurons,1,1)))
print("covar matrixes shape", covar_matrixes.shape)
print(covar_matrixes)
# inv_covar = covar_matrix.inverse().unsqueeze(0).repeat(2, 1, 1)
inv_covar = covar_matrixes.inverse()
print("inv_covar")
print(inv_covar.shape)
print(inv_covar)
# inv_covar_2 = covar_matrix.unsqueeze(0).repeat(2, 1, 1)
# print(inv_covar_2)

('covar shape', (3, 3))
[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]
('covar matrixes shape', torch.Size([2, 3, 3]))
tensor([[[1., 0., 0.],
         [0., 2., 0.],
         [0., 0., 3.]],

        [[1., 0., 0.],
         [0., 2., 0.],
         [0., 0., 3.]]], dtype=torch.float64)
inv_covar
torch.Size([2, 3, 3])
tensor([[[1.0000, 0.0000, -0.0000],
         [0.0000, 0.5000, -0.0000],
         [0.0000, 0.0000, 0.3333]],

        [[1.0000, 0.0000, -0.0000],
         [0.0000, 0.5000, -0.0000],
         [0.0000, 0.0000, 0.3333]]], dtype=torch.float64)


In [11]:
# check prod is identity
torch.matmul(covar_matrixes[0], inv_covar[0])

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=torch.float64)

In [12]:
# single vector case
d = distances.double()
print("d", d.shape)
print(d)
print("inv_covar", inv_covar.shape)
# print(inv_covar)
# de = torch.matmul(d,inv_covar)
de = torch.einsum('ijk, abk->ijb',d,inv_covar)
print("de", de.shape)
print(de)
### one input one neuron
# ded = torch.matmul(de,d)
### many inputs, one neuron
# ded = torch.einsum('ij,ij->i', de, d)
### many inputs, many neurons
ded = torch.einsum('ijk,ajk->ij', de, d)
print("ded", ded.shape)
print(ded)
# print(torch.matmul(de[0], d.transpose(0,1)))
r_activity_full = torch.exp(-ded)
print("full", r_activity_full.shape)
# print(r_activity_full)

('d', torch.Size([5, 2, 3]))
tensor([[[ 0.,  0.,  0.],
         [-3., -3., -3.]],

        [[ 3.,  3.,  3.],
         [ 0.,  0.,  0.]],

        [[ 6.,  6.,  6.],
         [ 3.,  3.,  3.]],

        [[ 9.,  9.,  9.],
         [ 6.,  6.,  6.]],

        [[12., 12., 12.],
         [ 9.,  9.,  9.]]], dtype=torch.float64)
('inv_covar', torch.Size([2, 3, 3]))
('de', torch.Size([5, 2, 3]))
tensor([[[ 0.,  0.,  0.],
         [-6., -3., -2.]],

        [[ 6.,  3.,  2.],
         [ 0.,  0.,  0.]],

        [[12.,  6.,  4.],
         [ 6.,  3.,  2.]],

        [[18.,  9.,  6.],
         [12.,  6.,  4.]],

        [[24., 12.,  8.],
         [18.,  9.,  6.]]], dtype=torch.float64)
('ded', torch.Size([5, 2]))
tensor([[   0., -165.],
        [ 330.,    0.],
        [ 660.,  165.],
        [ 990.,  330.],
        [1320.,  495.]], dtype=torch.float64)
('full', torch.Size([5, 2]))


In [13]:
A = np.arange(5*3*2).reshape(5,3,2)
B = np.arange(3*2*2).reshape(3,2,2)
print(A)
print(B)

[[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]

 [[12 13]
  [14 15]
  [16 17]]

 [[18 19]
  [20 21]
  [22 23]]

 [[24 25]
  [26 27]
  [28 29]]]
[[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]]


In [14]:
r1 = np.einsum('abc,ijc,klc->ab', A, B, A) 
print(r1.shape)
print(r1)

(5, 3)
[[  8100  36900  65700]
 [ 94500 123300 152100]
 [180900 209700 238500]
 [267300 296100 324900]
 [353700 382500 411300]]


In [15]:
(a,b,c) = np.shape(A)
(i,j,k) = np.shape(B)

ded = np.zeros(a*b).reshape(a,b)
# for every input in batch
for ai in range(a):
    # for every neuron
    for bi in range(b):
#         de =   np.matmul(distances[ai,bi,:], inv_covar_matrixes[bi,:,:])
#         ded[ai,bi] = np.matmul(de, np.transpose(distances[ai,bi,:]))
        ded[ai,bi] = np.einsum('i,ik,k->', A[ai,bi,:], B[bi,:,:],  A[ai,bi,:])

print(ded)
print(np.shape(ded))

[[3.0000e+00 1.4500e+02 7.8300e+02]
 [2.7300e+02 1.6150e+03 4.2210e+03]
 [9.7500e+02 4.6690e+03 1.0395e+04]
 [2.1090e+03 9.3070e+03 1.9305e+04]
 [3.6750e+03 1.5529e+04 3.0951e+04]]
(5, 3)


In [16]:
ded = np.einsum('...i,...ik,...k->...', A, B, A)
print(ded)

[[    3   145   783]
 [  273  1615  4221]
 [  975  4669 10395]
 [ 2109  9307 19305]
 [ 3675 15529 30951]]


In [17]:
a = torch.arange(6).reshape(2, 3)
print(a)
b = torch.arange(3)
print(b)
c = torch.einsum('k,ik->i', b, a)
print(c)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([0, 1, 2])
tensor([ 5, 14])


In [18]:
a = torch.eye(3)
print(a.shape)
print(a)

torch.Size([3, 3])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


In [19]:
b = torch.Tensor(10,3,3)
print(b.shape)


torch.Size([10, 3, 3])


In [20]:
np.prod(b.shape)

90

In [21]:
c = a.expand_as(b)
print(c.shape)

torch.Size([10, 3, 3])


In [22]:
print(c[0])

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


In [23]:
a = torch.tensor([1.8302, 1.3892, 1.0073, 1.4727, 1.6638, 1.8735, 1.7741, 1.2636, 1.8777,
        1.5461])

In [24]:
torch.tra

AttributeError: 'module' object has no attribute 'tra'

In [ ]:
sigmas = torch.nn.Parameter(torch.Tensor(10, 3, 3,), requires_grad=True)

In [ ]:
sigmas.data.copy_(sigmas)

In [ ]:
2.0*torch.eye(3).expand_as(sigmas)

In [ ]:
torch.einsum(equation=)